In [28]:
pip install -q -U tensorflow

Note: you may need to restart the kernel to use updated packages.


In [29]:
pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [30]:
import numpy as np
import pandas as pd
import random
import os
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import keras_tuner as kt

In [31]:
SEED = 42
EPOCHS = 500
BATCH_SIZE = 2048 
ACTIVATION = 'swish'
LEARNING_RATE = 0.0007

In [32]:
def seed_everything(seed=42):
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(SEED)

In [33]:
data = pd.read_csv('D:\CALCUL\python\coursera\ML\kaggle_nov21/train.csv')
test_data = pd.read_csv('D:\CALCUL\python\coursera\ML\kaggle_nov21/test.csv')

In [34]:
X = data.drop('target', axis = 1)
y = data['target']
del data

In [35]:
scaler = StandardScaler()
X = pd.DataFrame(columns=X.columns, data=scaler.fit_transform(X))
X_test = pd.DataFrame(columns=test_data.columns, data=scaler.transform(test_data))

In [36]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=SEED)

In [37]:
def model_builder(hp):
    model = keras.Sequential()
    inputs = keras.Input(shape=(X.shape[1]))
    
    x = layers.Dense(units=hp.Int("dense_01",
                                  min_value=32,
                                  max_value=256, 
                                  step=32),
                     activation='relu')(inputs)
    x = layers.Dropout(
        hp.Float('dense_dropout', min_value=0., max_value=0.9)
    )(x)
    
    num_block = hp.Int('num_block', min_value=1, max_value=4, step=1)
    
    for i in range(num_block):
        x = layers.Dense(units=hp.Int("units_" + str(i),
                                      min_value=32,
                                      max_value=256, 
                                      step=32),
                         activation='relu')(x)
        
        x = layers.Dropout(
          hp.Float('dense_dropout', min_value=0., max_value=0.9)
        )(x)
        
    outputs = keras.layers.Dense(1, activation="sigmoid")(x)
    
    model = keras.Model(inputs, outputs)
    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')
    model.compile(
        loss="binary_crossentropy",
        optimizer=keras.optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
        ),
        metrics=[roc_auc, "acc"]
    )
    model.summary()
    return model

In [38]:
tuner = kt.Hyperband(model_builder,
                    objective='val_acc',
                    max_epochs=10,
                    factor=3,
                    )

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 101)]             0         
                                                                 
 dense_4 (Dense)             (None, 32)                3264      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                1056      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,353
Trainable params: 4,353
Non-trainable p

In [39]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_acc', mode='max', patience=3)

In [40]:
tuner.search(x_train, y_train, 
             validation_split=0.2,
             epochs=100,
             callbacks=[stop_early],
             verbose=1, 
             )

Trial 30 Complete [00h 02m 41s]
val_acc: 0.7382444739341736

Best val_acc So Far: 0.7424222230911255
Total elapsed time: 00h 32m 04s
INFO:tensorflow:Oracle triggered exit


In [41]:
best_hps=tuner.get_best_hyperparameters()[0]

In [42]:
best_model = model_builder(best_hps)
print("="*20, " Best Model ", "="*20)
best_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 101)]             0         
                                                                 
 dense_4 (Dense)             (None, 256)               26112     
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                16448     
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 65        
                                                                 
Total params: 42,625
Trainable params: 42,625
Non-trainable

In [43]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name='val_acc', direction='max')
Trial summary
Hyperparameters:
dense_01: 256
dense_dropout: 0.7170491579980225
num_block: 1
units_0: 64
learning_rate: 0.0001
units_1: 64
units_2: 64
units_3: 256
tuner/epochs: 10
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.7424222230911255
Trial summary
Hyperparameters:
dense_01: 128
dense_dropout: 0.38007177263930875
num_block: 1
units_0: 192
learning_rate: 0.0001
units_1: 96
units_2: 96
units_3: 160
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 97a246dc35ac792da72315123a4164ca
Score: 0.7408555746078491
Trial summary
Hyperparameters:
dense_01: 64
dense_dropout: 0.19483306797912006
num_block: 2
units_0: 224
learning_rate: 0.001
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
units_1: 32
tuner/trial_id: ebe6dacdc16434dd6b1e2715dbeb6bce
Score: 0.7393666505813599
Trial summary
Hyperparameters:

In [45]:
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('dense_01')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.0001.



In [47]:
history = best_model.fit(x_train, y_train, validation_split=0.2, epochs=100)

Epoch 1/100
11250/11250 [==============================] - 16s 1ms/step - loss: 0.7114 - roc_auc: 0.5998 - acc: 0.5783 - val_loss: 0.6406 - val_roc_auc: 0.7400 - val_acc: 0.7134
Epoch 2/100
11250/11250 [==============================] - 16s 1ms/step - loss: 0.6358 - roc_auc: 0.7119 - acc: 0.6728 - val_loss: 0.6139 - val_roc_auc: 0.7448 - val_acc: 0.7285
Epoch 3/100
11250/11250 [==============================] - 16s 1ms/step - loss: 0.6222 - roc_auc: 0.7309 - acc: 0.6977 - val_loss: 0.6038 - val_roc_auc: 0.7456 - val_acc: 0.7329
Epoch 4/100
11250/11250 [==============================] - 16s 1ms/step - loss: 0.6146 - roc_auc: 0.7380 - acc: 0.7091 - val_loss: 0.5991 - val_roc_auc: 0.7460 - val_acc: 0.7342
Epoch 5/100
11250/11250 [==============================] - 17s 1ms/step - loss: 0.6083 - roc_auc: 0.7423 - acc: 0.7174 - val_loss: 0.5922 - val_roc_auc: 0.7463 - val_acc: 0.7382
Epoch 6/100
11250/11250 [==============================] - 19s 2ms/step - loss: 0.6021 - roc_auc: 0.7450 - acc

Epoch 47/100
11250/11250 [==============================] - 15s 1ms/step - loss: 0.5727 - roc_auc: 0.7588 - acc: 0.7443 - val_loss: 0.5695 - val_roc_auc: 0.7543 - val_acc: 0.7435
Epoch 48/100
11250/11250 [==============================] - 16s 1ms/step - loss: 0.5727 - roc_auc: 0.7588 - acc: 0.7443 - val_loss: 0.5695 - val_roc_auc: 0.7545 - val_acc: 0.7439
Epoch 49/100
11250/11250 [==============================] - 15s 1ms/step - loss: 0.5722 - roc_auc: 0.7596 - acc: 0.7443 - val_loss: 0.5688 - val_roc_auc: 0.7544 - val_acc: 0.7439
Epoch 50/100
11250/11250 [==============================] - 15s 1ms/step - loss: 0.5721 - roc_auc: 0.7595 - acc: 0.7444 - val_loss: 0.5690 - val_roc_auc: 0.7545 - val_acc: 0.7438
Epoch 51/100
11250/11250 [==============================] - 15s 1ms/step - loss: 0.5726 - roc_auc: 0.7591 - acc: 0.7444 - val_loss: 0.5693 - val_roc_auc: 0.7541 - val_acc: 0.7440
Epoch 52/100
11250/11250 [==============================] - 20s 2ms/step - loss: 0.5716 - roc_auc: 0.7604

Epoch 93/100
11250/11250 [==============================] - 24s 2ms/step - loss: 0.5698 - roc_auc: 0.7642 - acc: 0.7446 - val_loss: 0.5690 - val_roc_auc: 0.7551 - val_acc: 0.7439
Epoch 94/100
11250/11250 [==============================] - 24s 2ms/step - loss: 0.5701 - roc_auc: 0.7636 - acc: 0.7445 - val_loss: 0.5690 - val_roc_auc: 0.7552 - val_acc: 0.7437
Epoch 95/100
11250/11250 [==============================] - 29s 3ms/step - loss: 0.5697 - roc_auc: 0.7643 - acc: 0.7449 - val_loss: 0.5691 - val_roc_auc: 0.7548 - val_acc: 0.7436
Epoch 96/100
11250/11250 [==============================] - 20s 2ms/step - loss: 0.5701 - roc_auc: 0.7638 - acc: 0.7447 - val_loss: 0.5691 - val_roc_auc: 0.7547 - val_acc: 0.7439
Epoch 97/100
11250/11250 [==============================] - 21s 2ms/step - loss: 0.5694 - roc_auc: 0.7647 - acc: 0.7449 - val_loss: 0.5689 - val_roc_auc: 0.7549 - val_acc: 0.7439
Epoch 98/100
11250/11250 [==============================] - 22s 2ms/step - loss: 0.5696 - roc_auc: 0.7645

In [48]:
val_acc_per_epoch = history.history['val_acc']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f"Best epoch: {best_epoch}")

Best epoch: 43


In [56]:
skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
preds = []
scores = []
for fold, (idx_train, idx_valid) in enumerate(skf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    best_model = model_builder(best_hps)
    early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc",
                                                      mode='max',
                                                      verbose=1,
                                                      restore_best_weights=True,
                                                      patience=3)
    lr_scheduler_cb = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.2,
        patience=10,
        mode='min'
    )
    
    history = best_model.fit(X_train, y_train,
                validation_data = (X_valid, y_valid),
                batch_size = BATCH_SIZE, 
                epochs = best_epoch,
                callbacks = [early_stopping_cb, lr_scheduler_cb],
                )
    
    pred_valid = best_model.predict(X_valid)
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}" "\n")
    print('||'*30, "\n")
    
    test_preds = best_model.predict(X_test)
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 101)]             0         
                                                                 
 dense_34 (Dense)            (None, 256)               26112     
                                                                 
 dropout_23 (Dropout)        (None, 256)               0         
                                                                 
 dense_35 (Dense)            (None, 64)                16448     
                                                                 
 dropout_24 (Dropout)        (None, 64)                0         
                                                                 
 dense_36 (Dense)            (None, 1)                 65        
                                                                 
Total params: 42,625
Trainable params: 42,625
Non-trainabl

Epoch 38/43
235/235 [==============================] - 3s 11ms/step - loss: 0.6034 - roc_auc: 0.7454 - acc: 0.7247 - val_loss: 0.5894 - val_roc_auc: 0.7485 - val_acc: 0.7406 - lr: 1.0000e-04
Epoch 39/43
235/235 [==============================] - 3s 12ms/step - loss: 0.6021 - roc_auc: 0.7459 - acc: 0.7253 - val_loss: 0.5875 - val_roc_auc: 0.7486 - val_acc: 0.7408 - lr: 1.0000e-04
Epoch 40/43
235/235 [==============================] - 3s 11ms/step - loss: 0.6012 - roc_auc: 0.7462 - acc: 0.7263 - val_loss: 0.5867 - val_roc_auc: 0.7486 - val_acc: 0.7411 - lr: 1.0000e-04
Epoch 41/43
235/235 [==============================] - 3s 11ms/step - loss: 0.6002 - roc_auc: 0.7465 - acc: 0.7273 - val_loss: 0.5855 - val_roc_auc: 0.7487 - val_acc: 0.7412 - lr: 1.0000e-04
Epoch 42/43
235/235 [==============================] - 3s 11ms/step - loss: 0.5986 - roc_auc: 0.7474 - acc: 0.7282 - val_loss: 0.5845 - val_roc_auc: 0.7487 - val_acc: 0.7414 - lr: 1.0000e-04
Epoch 43/43
235/235 [========================

235/235 [==============================] - 3s 12ms/step - loss: 0.6206 - roc_auc: 0.7337 - acc: 0.7017 - val_loss: 0.6034 - val_roc_auc: 0.7500 - val_acc: 0.7362 - lr: 1.0000e-04
Epoch 25/43
235/235 [==============================] - 3s 12ms/step - loss: 0.6198 - roc_auc: 0.7345 - acc: 0.7037 - val_loss: 0.6033 - val_roc_auc: 0.7500 - val_acc: 0.7369 - lr: 1.0000e-04
Epoch 26/43
235/235 [==============================] - 3s 12ms/step - loss: 0.6184 - roc_auc: 0.7362 - acc: 0.7059 - val_loss: 0.6028 - val_roc_auc: 0.7501 - val_acc: 0.7370 - lr: 1.0000e-04
Epoch 27/43
235/235 [==============================] - 3s 12ms/step - loss: 0.6175 - roc_auc: 0.7370 - acc: 0.7070 - val_loss: 0.6013 - val_roc_auc: 0.7501 - val_acc: 0.7377 - lr: 1.0000e-04
Epoch 28/43
235/235 [==============================] - 3s 12ms/step - loss: 0.6167 - roc_auc: 0.7379 - acc: 0.7086 - val_loss: 0.6009 - val_roc_auc: 0.7501 - val_acc: 0.7379 - lr: 1.0000e-04
Epoch 29/43
235/235 [==============================] - 3s

235/235 [==============================] - 3s 11ms/step - loss: 0.6428 - roc_auc: 0.6989 - acc: 0.6584 - val_loss: 0.6254 - val_roc_auc: 0.7481 - val_acc: 0.7243 - lr: 1.0000e-04
Epoch 11/43
235/235 [==============================] - 3s 11ms/step - loss: 0.6393 - roc_auc: 0.7059 - acc: 0.6660 - val_loss: 0.6210 - val_roc_auc: 0.7487 - val_acc: 0.7266 - lr: 1.0000e-04
Epoch 12/43
235/235 [==============================] - 3s 11ms/step - loss: 0.6371 - roc_auc: 0.7101 - acc: 0.6710 - val_loss: 0.6185 - val_roc_auc: 0.7492 - val_acc: 0.7290 - lr: 1.0000e-04
Epoch 13/43
235/235 [==============================] - 3s 11ms/step - loss: 0.6342 - roc_auc: 0.7140 - acc: 0.6749 - val_loss: 0.6167 - val_roc_auc: 0.7495 - val_acc: 0.7297 - lr: 1.0000e-04
Epoch 14/43
235/235 [==============================] - 3s 11ms/step - loss: 0.6322 - roc_auc: 0.7180 - acc: 0.6804 - val_loss: 0.6142 - val_roc_auc: 0.7498 - val_acc: 0.7313 - lr: 1.0000e-04
Epoch 15/43
235/235 [==============================] - 3s

In [58]:
submission = pd.read_csv('D:\CALCUL\python\coursera\ML\kaggle_nov21/sample_submission.csv')
predictions = np.mean(preds, axis=0)

submission['target'] = predictions
submission.to_csv('./submission_Nov21_keras_tuned1.csv', index=False)

In [59]:
submission.head()

,id,target
0,600000,0.694246
1,600001,0.691218
2,600002,0.753260
3,600003,0.434063
4,600004,0.673644
